
# Load data

**source**:  Airbnb data for Athens at http://insideairbnb.com/get-the-data

**destination**: airbnb.raw schema 

In [0]:
%run ./setup

In [0]:
import io
import pandas as pd
import requests

def load_data_from_url_as_spark_df(url, **params):
    """Loads data from url. Optional keyword arguments are passed to pandas.read_csv."""
    response = requests.get(url)
    dx = pd.read_csv(io.BytesIO(response.content), **params)  
    return spark.createDataFrame(dx)

In [0]:
import json 
import pyspark.sql.functions as f
from pyspark.sql.types import StringType, TimestampType
from delta.tables import DeltaTable

def update_schema_with_metadata_fields(schema):
    """Helper method to add metadata fields to contact schema."""
    return schema\
            .add("processing_datetime", TimestampType(), True)\
            .add("area", StringType(), True)
        
def add_metadata_columns(df, area: str):
    """Helper method to add metadata columns to dataframe."""
    return (
        df.withColumn("processing_datetime", f.current_timestamp())
        .withColumn("area", f.lit(area))
    )


## Load Listings

In [0]:
with open(contracts_path + "/listing_schema.json", "r") as fin:
    listing_schema = StructType.fromJson(json.loads(fin.read()))

raw_listing_schema = update_schema_with_metadata_fields(listing_schema)

listing_delta = (
    DeltaTable.createIfNotExists(spark)
    .addColumns(raw_listing_schema)
    .tableName('airbnb.raw.listings')
).execute()

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File <command-1242831895003030>, line 1
----> 1 with open(contracts_path + "/listing_schema.json", "r") as fin:
      2     listing_schema = StructType.fromJson(json.loads(fin.read()))
      4 raw_listing_schema = update_schema_with_metadata_fields(listing_schema)

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:324, in _modified_open(file, *args, **kwargs)
    317 if file in {0, 1, 2}:
    318     raise ValueError(
    319         f"IPython won't let you open fd={file} by default "
    320         "as it is likely to crash IPython. If you know what you are doing, "
    321         "you can use builtins' open."
    322     )
--> 324 return io_open(file, *args, **kwargs)

FileNotFoundError: [Errno 2] No such file or directory: '/Workspace/Shared/MissingValue/listing_schema.json'

In [0]:
listings_url = "http://data.insideairbnb.com/greece/attica/athens/2023-09-21/data/listings.csv.gz"

listings_df = load_data_from_url_as_spark_df(
    listings_url, sep=',', index_col=0, quotechar='"', compression='gzip'
)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
_ = (listings_df
     .transform(lambda x: add_metadata_columns(x, "Athens"))
     .writeTo("airbnb.raw.listings")
     .append())

Contracts path : contrancts_path="/Workspace/Shared/MissingValue"


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1242831895003032>, line 1
----> 1 _ = (listings_df
      2      .transform(lambda x: add_metadata_columns(x, "Athens"))
      3      .writeTo("airbnb.raw.listings")
      4      .append())

NameError: name 'listings_df' is not defined